# Location

The `location` directive, used within the `server` context, specifies how NGINX should process requests based on the URI.

More in [correspoding section](http://nginx.org/en/docs/http/ngx_http_core_module.html#location) of the official documentation.

In [1]:
import os
import docker
import requests

docker_client = docker.DockerClient()

## Setup

It turns out that it's quite a complex task to build examples that show how everything works, so this section describes what we need to show everything. In summary, we need

- Proxied server - server where we'll redirect requests to nginx.
- And nginx, which can be configured differently for different examples.

### Network

We need a way to connect containers between each other - so we're going to create a network that will be used for container communication.

In [2]:
network = docker_client.networks.create(name = "test_network")

### Proxied server

As a proxy server, we'll use httpbin - an http pseudo-server that allows us to study the requests sent to it.

In [3]:
client_container = docker_client.containers.run(
    image = "kennethreitz/httpbin",
    detach = True,
    remove = True,
    name="client_container",
    network = network.name
)

### Nginx

Now we configure nginx. It's config can be changed, but by default it will forward requests to it root to the http server we created earlier.

In [4]:
%%writefile location_files/nginx.conf
events {}
http {
    server {
        listen 80;
        location / {proxy_pass "http://client_container";}
    }
}

Overwriting location_files/nginx.conf


And we run docker container with nginx on the same network as our client http server.

In [5]:
nginx_container = docker_client.containers.run(
    image = "nginx",
    volumes = {
        f"{os.getcwd()}/location_files/nginx.conf": 
        {'bind': '/etc/nginx/nginx.conf', 'mode': 'rw'}
    },
    ports = {80: 80},
    detach = True,
    remove = True,
    network = network.name
)

### Clear environment

To avoid a lot of rubbish in your environment, you need to delete the containers you have created.

In [6]:
client_container.stop()
nginx_container.stop()
network.remove()

## Headers to server (`proxy_set_header`)

Allows redefining or appending fields to the request header passed to the proxied server. You can add aditional headers to the http request that will be sent to the destination server. So by using syntax `proxy_set_header <header field> <value>;`.

The following example changes the nginx config to add two new fields to the http header `Name` and `SecondName`.

In [6]:
%%writefile location_files/nginx.conf
events {}
http {
    server {
        listen 80;
        location / {
            proxy_pass "http://client_container/headers";
            proxy_set_header Name Fedor;
            proxy_set_header SecondName Kobak;
        }
    }
}

Overwriting location_files/nginx.conf


After reloading nginx, we need to try sending the request to the original server and then to nginx.

In [7]:
ans = nginx_container.exec_run("nginx -s reload")

print("=====Original request=====")
print(
    requests.get("http://localhost/headers").content.decode("utf-8")
)
print("=====Proxied request=====")
print(
    requests.get("http://localhost:80").content.decode("utf-8")
)

=====Original request=====
{
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Connection": "close", 
    "Host": "client_container", 
    "User-Agent": "python-requests/2.31.0"
  }
}

=====Proxied request=====
{
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Connection": "close", 
    "Host": "client_container", 
    "Name": "Fedor", 
    "Secondname": "Kobak", 
    "User-Agent": "python-requests/2.31.0"
  }
}



If you compare the output of the original and proxied requests, you can see that the proxied requests have additional headers - just as we specified in the nginx configuration.